# futureEXPERT check-in options

The [getting started notebook](./getting_started.ipynb) shows the most simple approach to check in time series for futureEXPERT. In this notebook you can find various ways to check in your data.

## Prerequisites for running this notebook
Login with your account and create a forecasting configuration. For details and configuration options please have a look at the [getting started notebook](./getting_started.ipynb) or other more advanced notebooks.

In [ ]:
from futureexpert import *

client = ExpertClient.from_dotenv()
fc_report_config = ReportConfig(title='My first report',
                                forecasting=ForecastingConfig(fc_horizon=7),
                                preprocessing=PreprocessingConfig(),
                                method_selection=MethodSelectionConfig(number_iterations=8))

## Option 1: Check-in your time series via the _future_ frontend

Check in your time series for futureEXPERT in the _future_ frontend https://future.prognostica.de:

1. Upload your data using Python or the future frontend.
2. Navigate to 'My Data' in future frontend.
3. Start the futureCHECK-IN process from a file.
4. Follow along the futureCHECK-IN steps in the frontend.
5. Click on 'Continue with...' and 'Forecast via EXPERT' and copy the version ID.

You can then use your checked-in data for forecasting in futureEXPERT.

In [ ]:
ts_version = 'TBD' # replace with your version

# Request forecast creation and retrieve the id for status requests and result retrieval
forecast_identifier_from_version_id = client.start_forecast(version=ts_version, config=fc_report_config)

### Option 2: Use the check-in configuration file from the _future_ frontend

#### Get a check-in configuration file

Use a check-in configuration file generated during the CHECK-IN process in the _future_ frontend https://future.prognostica.de:

1. Upload your data using Python or the future frontend.
2. Navigate to 'My Data' in future frontend.
3. Start the futureCHECK-IN process from a file.
4. Follow along the futureCHECK-IN steps in the frontend.
5. Download the configuration at the end.
6. Adjust the file pathe in the below code block.

You can then reuse your uploaded data with the downloaded futureCHECK-IN configuration for forecasting in futureEXPERT or even reuse the futureCHECK-IN configuration for new data.

#### Invoke forecast computation




In [ ]:
# Request forecast creation and retrieve the id for status requests and result retrieval
forecast_identifier_from_config_file = client.start_forecast_from_raw_data(raw_data_source='../example-data/bicycle_data.csv',
                                                                           config_fc=fc_report_config,
                                                                           config_checkin='../example-data/config_bicycle_data.json')

### Option 3: Configure check-in in code

Configure the data definition and time series creation in code.

**Note:** The prefered way to configure the data defintion is using the _future_ frontend at https://future.prognostica.de, as it provides the full range of available features, e.g. rules for handling detected data issues, and inclusion and exclusion rules for data preparation.

In [3]:
data_definition = DataDefinition(
    date_columns=DateColumn(name='Date', format='%Y-%m-%d'),
    value_columns=[ValueColumn(name='value')],
    group_columns=[GroupColumn(name='Name')])

ts_creation_config = TsCreationConfig(time_granularity='daily',
                                      grouping_level=['Name'],
                                      value_columns_to_save=['value'],
                                      missing_value_handler='setToZero')


# Request forecast creation and retrieve the id for status requests and result retrieval
forecast_identifier_from_code_definition = client.start_forecast_from_raw_data(raw_data_source='../example-data/bicycle_data.csv',
                                                                               data_definition=data_definition,
                                                                               config_ts_creation=ts_creation_config,
                                                                               config_fc=fc_report_config,
                                                                               file_specification=FileSpecification(delimiter=';', decimal='.'))

## Inspect the forecasting results

In this case, we first wait for all forecasting results before accessing the results.

In [ ]:
import time

# Watch the current status of the forecasting report
while not all(client.get_report_status(id=forecast_identifier).is_finished for forecast_identifier in [forecast_identifier_from_version_id, forecast_identifier_from_config_file, forecast_identifier_from_code_definition]):
    print('Waiting another 30 seconds to finish forecasting...')
    time.sleep(30)  # Wait between status requests

# Retrieve the final results
results_from_version_id = client.get_fc_results(
    id=forecast_identifier_from_version_id, include_backtesting=True, include_k_best_models=2)
results_from_config_file = client.get_fc_results(
    id=forecast_identifier_from_config_file, include_backtesting=True, include_k_best_models=2)
results_from_code_definition = client.get_fc_results(
    id=forecast_identifier_from_code_definition, include_backtesting=True, include_k_best_models=2)